https://zhuanlan.zhihu.com/p/75206669

CLASS torch.nn.Sequential(*args)
A sequential container. Modules will be added to it in the order they are passed in the constructor. Alternatively, an ordered dict of modules can also be passed in.

CLASS torch.nn.ModuleList(modules=None)
Holds submodules in a list.

ModuleList can be indexed like a regular Python list, but modules it contains are properly registered, and will be visible by all Module methods.



## nn.Sequential
nn.Sequential里面的模块按照顺序进行排列的，所以必须确保前一个模块的输出大小和下一个模块的输入大小是一致的。如下面的例子所示：

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class net_seq(nn.Module):
    def __init__(self):
        super(net_seq, self).__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(1,20, 5),
            nn.ReLU(),
            nn.Conv2d(20, 64, 5),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.seq(x)
    
net_seq = net_seq()
print(net_seq)

net_seq(
  (seq): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
  )
)


nn.Sequential中可以使用OrderedDict来指定每个module的名字，而不是采用默认的命名方式(按序号 0,1,2,3...)。例子如下：

In [3]:
from collections import OrderedDict

class net_seq(nn.Module):
    def __init__(self):
        super(net_seq, self).__init__()
        self.seq = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(1, 20, 5)),
            ('relu1', nn.ReLU()),
            ('conv2', nn.Conv2d(20, 64, 5)),
            ('relu2', nn.ReLU())
        ]))
    
    def forward(self, x):
        return self.seq(x)
    
net_seq = net_seq()
print(net_seq)

net_seq(
  (seq): Sequential(
    (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU()
    (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
  )
)


## nn.ModuleList

它是一个存储不同的Module，并自动将每个module的parameters添加到网络之中的容器。你可以把任意的nn.Module的子类加到这个list里面，方法和Python自带的list一样，无非是extend， append等操作。但不同于一半的list，加入到nn.ModuleList里面的module会自动注册到整个网络上的，同时module的parameters也会自动添加到整个网络中。

## nn.Sequential与nn.ModuleList的区别

### 不同点1：

nn.Sequential内部实现了forward函数。而nn.ModuleList则没有实现forward函数。也就是说nn.Sequential是一个完整的网络。而nn.ModuleList只是一个包含不同模块的容器而已。



In [4]:
# 对于nn.Sequential

#例1：这是来自官方文档的例子
seq = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )
print(seq)

input = torch.randn(16, 1, 20, 20)
print(seq(input))

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)
tensor([[[[0.0000e+00, 0.0000e+00, 6.4640e-02,  ..., 1.1690e-01,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 1.1823e-02,  ..., 0.0000e+00,
           6.5997e-02, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 2.1376e-02,  ..., 3.4557e-01,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 1.5528e-02, 7.5280e-02,  ..., 2.1073e-01,
           1.7520e-01, 0.0000e+00],
          [0.0000e+00, 1.5166e-01, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           1.6569e-01, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 9.3210e-02],
          [7.4425e-02, 0.0000e+00, 0.0000e+00,  ..., 1.7898e-02,
           0.0000e+00, 2.3116e-01],
          [0.0000e+0

In [6]:
# 对于nn.ModuleList：
#例1：若按照下面这么写，则会产生错误
modlist = nn.ModuleList([
         nn.Conv2d(1, 20, 5),
         nn.ReLU(),
         nn.Conv2d(20, 64, 5),
         nn.ReLU()
         ])
print(modlist)
input = torch.randn(16, 1, 20, 20)
print(modlist(input))

ModuleList(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)


TypeError: forward() takes 1 positional argument but 2 were given

In [8]:
#例2：写出forward函数

class net2(nn.Module):
    def __init__(self):
        super(net2, self).__init__()
        self.modlist = nn.ModuleList([
            nn.Conv2d(1, 20, 5),
            nn.ReLU(),
            nn.Conv2d(20, 64, 5),
            nn.ReLU()
        ])
        
    def forward(self, x):
        #return self.modlist(x) #这种写法就是nn.Sequential的写法，是会报错的
        #如下利用for循环的方式是正确的方式
        for m in self.modlist:
            x = m(x)
        return x
    
input_tensor = torch.randn(16, 1, 20, 20)
net2 = net2()
print(net2(input_tensor).shape)

torch.Size([16, 64, 12, 12])


如果完全直接用 nn.Sequential，确实是可以的，但这么做的代价就是失去了部分灵活性，不能自己去定制 forward 函数里面的内容了。

一般情况下 nn.Sequential 的用法是来组成卷积块 (block)，然后像拼积木一样把不同的 block 拼成整个网络，让代码更简洁，更加结构化。

## 不同点2：
nn.Sequential可以使用OrderedDict对每层进行命名，上面已经阐述过了；

## 不同点3：
### nn.Sequential里面的模块按照顺序进行排列的，所以必须确保前一个模块的输出大小和下一个模块的输入大小是一致的。
### 而nn.ModuleList并没有定义一个网络，它只是将不同的模块存储在一起，这些模块之间并没有先后顺序可言。见下面代码：


In [11]:
class net3(nn.Module):
    def __init__(self):
        super(net3, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 20), nn.Linear(20, 30), nn.Linear(5, 10)])
    def forward(self, x):
        x = self.linears[2](x)
        x = self.linears[0](x)
        x = self.linears[1](x)
        
        return x
    
net3 = net3()
print(net3)

input_tensor = torch.randn(32, 5)
print(net3(input_tensor).shape) #torch.Size([32, 30])

## 根据net3的结果，可以看出这个ModuleList里面的顺序不能决定什么，网络的执行顺序是由forward函数来决定的。
## 为了使代码的可读性好一些，最好把ModuleList和forward中的顺序保持一致。

net3(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=30, bias=True)
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
)
torch.Size([32, 30])


### 不同点4
有的时候网络中有很多相似或者重复的层，我们一般会考虑用 for 循环来创建它们，而不是一行一行地写，比如：
layers = [nn.Linear(10, 10) for i in range(5)]
那么这里我们使用ModuleList：



In [13]:
class net4(nn.Module):
    def __init__(self):
        super(net4, self).__init__()
        layers = [nn.Linear(10, 10) for i in range(5)]
        self.linears = nn.ModuleList(layers)
    def forward(self, x):
        for layer in self.linears:
            x = layers(x)
        return x
    
net = net4()
print(net)

net4(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)


## 参考阅读
https://zhuanlan.zhihu.com/p/64990232

In [ ]:
一般情况下 nn.Sequential 的用法是来组成卷积块 (block)，然后像拼积木一样把不同的 block 拼成整个网络，让代码更简洁，更加结构化。

## 总结
本文中我们通过一些实例学习了 ModuleList 和 Sequential 这两种 nn containers，ModuleList 就是一个储存各种模块的 list，这些模块之间没有联系，没有实现 forward 功能，但相比于普通的 Python list，ModuleList 可以把添加到其中的模块和参数自动注册到网络上。而Sequential 内的模块需要按照顺序排列，要保证相邻层的输入输出大小相匹配，内部 forward 功能已经实现，可以使代码更加整洁。在不同场景中，如果二者都适用，那就看个人偏好了。非常推荐大家看一下 PyTorch 官方的 TorchVision 下面模型实现的代码，能学到很多构建网络的技巧。
